In [41]:
from google.colab import drive
import zipfile
import os

# Google Drive'ı bağla
drive.mount('/content/drive')

# Dataset 1: Fresh_and_Rotten_Classification
zip_path1 = "/content/drive/MyDrive/Colab Notebooks/Fresh_and_Rotten_Classification.zip"
extract_path1 = "/content/Fresh_and_Rotten_Classification"

if not os.path.exists(extract_path1):
    with zipfile.ZipFile(zip_path1, 'r') as zip_ref:
        zip_ref.extractall(extract_path1)
    print(f"dataset1 çıkartıldı: {extract_path1}")
else:
    print(f"dataset1 zaten mevcut, tekrar çıkartılmadı: {extract_path1}")

# Dataset 2: Fruit_And_Vegetable_Diseases_Dataset
zip_path2 = "/content/drive/MyDrive/Colab Notebooks/Fruit_And_Vegetable_Diseases_Dataset.zip"
extract_path2 = "/content/Fruit_And_Vegetable_Diseases_Dataset"

if not os.path.exists(extract_path2):
    with zipfile.ZipFile(zip_path2, 'r') as zip_ref:
        zip_ref.extractall(extract_path2)
    print(f"dataset2 çıkartıldı: {extract_path2}")
else:
    print(f"dataset2 zaten mevcut, tekrar çıkartılmadı: {extract_path2}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dataset1 zaten mevcut, tekrar çıkartılmadı: /content/Fresh_and_Rotten_Classification
dataset2 zaten mevcut, tekrar çıkartılmadı: /content/Fruit_And_Vegetable_Diseases_Dataset


In [42]:
import tensorflow as tf

def find_tf_bozuk_gorseller(root_dir):
    bozuklar = []

    for class_dir in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_dir)
        if not os.path.isdir(class_path):
            continue

        for file in os.listdir(class_path):
            file_path = os.path.join(class_path, file)
            try:
                img_raw = tf.io.read_file(file_path)
                _ = tf.image.decode_image(img_raw)  # TensorFlow'un kendi decoder'ı
            except Exception as e:
                print(f"TF bozuk: {file_path} - {e}")
                bozuklar.append(file_path)

    print(f"\nTensorFlow tarafından okunamayan dosya sayısı: {len(bozuklar)}")
    for f in bozuklar:
      try:
          os.remove(f)
          print(f"Silindi: {f}")
      except Exception as e:
          print(f"Silinemedi: {f} - {e}")

    return bozuklar

#bozuklar = find_tf_bozuk_gorseller("/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Test")
#bozuklar = find_tf_bozuk_gorseller("/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Train")


In [43]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.feature import hog
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [80]:
def extract_features(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None

    img = cv2.resize(img, (128, 128))  # sabit boyut

    # HSV Histogram
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv_hist = cv2.calcHist([hsv], [0, 1, 2], None,
                            [4, 4, 4],
                            [0, 180, 0, 256, 0, 256]).flatten()

    # HOG Özelliği
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hog_feat = hog(
        gray,
        orientations=6,
        pixels_per_cell=(16, 16),
        cells_per_block=(2, 2),
        block_norm='L2-Hys',
        feature_vector=True
    )

    return np.concatenate((hsv_hist, hog_feat))


In [45]:
def load_dataset(root_path):
    x, y = [], []
    for label in tqdm(os.listdir(root_path)):
        class_path = os.path.join(root_path, label)
        if not os.path.isdir(class_path):
            continue
        for fname in os.listdir(class_path):
            fpath = os.path.join(class_path, fname)
            feat = extract_features(fpath)
            if feat is not None:
                x.append(feat)
                y.append(label)
    return np.array(x), np.array(y)


In [46]:
# Dataset 1: Fresh vs Rotten Dataset
print("Fresh vs Rotten Dataset")
train_dir1 = "/content/Fresh_and_Rotten_Classification/Fresh_and_Rotten_Classification/Train"
test_dir1 = "/content/Fresh_and_Rotten_Classification/Fresh_and_Rotten_Classification/Test"

x_train1, y_train1 = load_dataset(train_dir1)
x_test1, y_test1 = load_dataset(test_dir1)

print("Eğitim (input:vektör):", x_train1.shape)
print("Test (input:vektör):", x_test1.shape)

# Dataset 2: Fruit and Vegetable Diseases Dataset
print("\nFruit and Vegetable Diseases Dataset")
train_dir2 = "/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Train"
test_dir2 = "/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Test"

x_train2, y_train2 = load_dataset(train_dir2)
x_test2, y_test2 = load_dataset(test_dir2)

print("Eğitim (input:vektör):", x_train2.shape)
print("Test (input:vektör):", x_test2.shape)


Fresh vs Rotten Dataset


100%|██████████| 15/15 [00:46<00:00,  3.08s/it]


Eğitim (input:vektör): (21044, 1240)
Test (input:vektör): (6738, 1240)

Fruit and Vegetable Diseases Dataset


100%|██████████| 29/29 [00:49<00:00,  1.70s/it]

Eğitim (input:vektör): (23425, 1240)
Test (input:vektör): (5849, 1240)


In [73]:
# PCA ile boyut indirgeme
pca1 = PCA(n_components=50)
x_train1_pca = pca1.fit_transform(x_train1)
x_test1_pca = pca1.transform(x_test1)

print("X_train boyutu:", x_train1_pca.shape)
print("X_test boyutu:", x_test1_pca)


pca2 = PCA(n_components=50)
x_train2_pca = pca2.fit_transform(x_train2)
x_test2_pca = pca2.transform(x_test2)

print("X_train boyutu:", x_train2_pca.shape)
print("X_test boyutu:", x_test2_pca)


X_train boyutu: (21044, 50)
X_test boyutu: [[-1.58273080e+03 -3.56936226e+02 -1.81018248e+02 ...  3.13424809e+01
   9.84864515e+01  1.99683048e+01]
 [ 6.26217677e+02 -9.15709609e+02  6.24785048e+02 ... -1.23472354e+00
   2.99830691e+00 -1.15053560e+01]
 [-1.05929127e+03 -9.70977283e+02 -8.32926350e+01 ... -2.26937380e+00
  -2.75012104e+00  4.30127910e+00]
 ...
 [-1.29246402e+03  4.70361750e+03  7.83896315e+02 ...  3.37334018e+00
  -1.10705266e+00 -6.25032697e+00]
 [-1.27431065e+03  1.92144429e+03  7.33659080e+02 ...  2.14850002e+00
  -7.70954349e-01 -2.33619332e+00]
 [-1.72252891e+03  2.14429712e+03  5.75336194e+02 ...  5.13355895e+00
  -3.53242818e-01 -1.45995517e+00]]
X_train boyutu: (23425, 50)
X_test boyutu: [[ 1.75403072e+03 -1.08712514e+03 -8.41448428e+02 ... -8.70137935e+00
  -1.45228854e-01 -2.10189101e-01]
 [ 2.04197602e+03 -1.54870180e+03  7.74905027e+02 ... -1.98678269e+01
  -4.63655109e-01  1.27546672e+00]
 [ 4.58772200e+03 -2.44220858e+02 -9.83915038e+02 ... -1.39873971e+0

In [64]:
def create_rf_model(n_estimators=100, max_depth=None, random_state=42):
    return RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=random_state,
        n_jobs=-1,
        verbose=1
    )

In [78]:
import pickle

# Model 1: Fresh vs Rotten Dataset
model1 = create_rf_model(n_estimators=100, max_depth=None)
model1.fit(x_train1_pca, y_train1)
y_pred1 = model1.predict(x_test1_pca)

with open("rf_model_freshrotten.pkl", "wb") as f:
    pickle.dump(model1, f)

# Model 2: Fruit and Vegetable Dataset
model2 = create_rf_model(n_estimators=100, max_depth=None)
model2.fit(x_train2_pca, y_train2)
y_pred2 = model2.predict(x_test2_pca)

with open("rf_model_disease.pkl", "wb") as f:
    pickle.dump(model2, f)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.7s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


In [79]:
print("Dataset 1: Fresh vs Rotten Dataset")
print("Doğruluk:", accuracy_score(y_test1, y_pred1))
print(classification_report(y_test1, y_pred1))

print("\nDataset 2: Fruit and Vegetable Dataset")
print("Doğruluk:", accuracy_score(y_test2, y_pred2))
print(classification_report(y_test2, y_pred2))



Dataset 1: Fresh vs Rotten Dataset
Doğruluk: 0.9964381121994658
                precision    recall  f1-score   support

   freshapples       1.00      0.99      1.00       791
   freshbanana       0.99      1.00      0.99       892
 freshcucumber       1.00      1.00      1.00       279
     freshokra       1.00      1.00      1.00       370
  freshoranges       0.99      0.99      0.99       388
   freshpotato       1.00      1.00      1.00       270
   freshtomato       1.00      1.00      1.00       255
  rottenapples       1.00      0.99      0.99       988
  rottenbanana       1.00      1.00      1.00       900
rottencucumber       1.00      1.00      1.00       255
    rottenokra       1.00      1.00      1.00       224
 rottenoranges       0.99      0.98      0.99       403
  rottenpotato       1.00      1.00      1.00       370
  rottentomato       1.00      1.00      1.00       353

      accuracy                           1.00      6738
     macro avg       1.00      1.00   